In [1]:
%matplotlib inline

import sys 
import os 

nb_dir = os.getcwd()
if nb_dir not in sys.path:
    sys.path.append(nb_dir)

from plotlib.loaders import *
from plotlib.plotters import *

from phdconf import config 
from phdconf.config import *

In [2]:
queries = load_queries(config.AUS_TOPIC_PATH)
broad, specific = load_query_types(queries)

In [3]:
BASE_DIR = os.path.join(os.environ["HOME"], "phd-generated")

In [4]:
index_name = 'filtered-phrasestop'#, config.SIGIR_INDEX_NAME]
qrel_paths =  [config.AUS_QREL_PATH]#, config.SIGIR_QREL_PATH]
rel_levels = [config.AUS_REL_LEVEL]#, config.SIGIR_REL_LEVEL]
display_names = ['AUS']#, 'SIGIR']

In [5]:
base_df = load_1d_dfs([index_name], qrel_paths, os.path.join('/home/danlocke/phd-generated', 'preprocessing', 'dirichlet_prior'),'case-topics-{0}-unigram_dir_mu_{1:.2f}.run', rel_levels, 1050, 1050, 1)[0][0]
base_qry = load_1d_dfs([index_name], qrel_paths, os.path.join('/home/danlocke/phd-generated', 'preprocessing', 'dirichlet_prior'),'case-topics-{0}-unigram_dir_mu_{1:.2f}.run', rel_levels, 1050, 1050, 1, per_query=True)[0][0]

## Phrase scoring

In [6]:
names = ['nnphrase', 'nnphrase-exp', 'phrase-exp-qry', 'nnphrase-exp-qry']
inter = Interpolater(os.path.join(BASE_DIR, 'preprocessing', 'dirichlet_prior', 'case-topics-filtered-phrasestop-unigram_dir_mu_1050.00.run'), normalize=True)

_dir = '/home/danlocke/go/src/github.com/dan-locke/phd/phrase-expansion/'

In [7]:
dfs = []
for n in names: 
    interped_dfs = []
    for _lambda in np.arange(0, 1.0, 0.01):
        inter.interpolate(os.path.join(_dir, 'case-topics-{0}-{1}.run'.format(index_name, n)), _lambda, 'tmp.run')
        interped_dfs.append(load_dfs(config.AUS_QREL_PATH, config.AUS_REL_LEVEL, '', ['tmp.run'])[0])
    dfs.append(interped_dfs)

In [8]:
qry_dfs = []
for n in names: 
    interped_dfs = []
    for _lambda in np.arange(0, 1.0, 0.01):
        inter.interpolate(os.path.join(_dir, 'case-topics-{0}-{1}.run'.format(index_name, n)), _lambda, 'tmp.run')
        interped_dfs.append(load_dfs(config.AUS_QREL_PATH, config.AUS_REL_LEVEL, '', ['tmp.run'], per_query=True)[0])
    qry_dfs.append(interped_dfs)

In [9]:
metrics = copy.copy(config.METRIC_NAMES)
del metrics['recall_100']

In [10]:


to = 50

plot = plot_tune_1d_comp(['base', 'phrase', 'phrase-exp', 'phrase-qry', 'phrase-exp-qry'], metrics, [[base_df for x in range(to+1)]] + [x[:to+1] for x in dfs], 0.00, (to)/100, 0.01, legend_x=0.991, styles=['--'], ylims=ylims)
plot.savefig('figures/ausnl-phrase-expansion-interp.pdf')

<Figure size 1152x432 with 6 Axes>

In [11]:
# base_sdm = load_1d_dfs([index_name], qrel_paths, os.path.join('/home/danlocke/phd-generated', 'sdm_rerank'), 'case-topics-filtered-phrasestop-sdm_rerank-smooth-dir-mu-{1:.2f}-weights-0.80-0.05-0.15-window-6.run', rel_levels, 1050, 1050, 1)[0][0]

plot = plot_tune_1d_comp(['base', 'phrase'], metrics, [[base_df for x in range(to+1)]] + [dfs[0][:to+1]], 0.00, (to)/100, 0.01, legend_x=0.965, styles=['--'])
plot.savefig('figures/ausnl-phrase-interp.pdf')

<Figure size 1152x432 with 6 Axes>

In [12]:
tt_folds = read_folds('ausnl-folds.txt')

In [13]:
stop_df = pd.DataFrame(columns=metrics)

for ab, runs in zip(['phrase', 'phrase-exp', 'phrase-qry', 'phrase-exp-qry'], qry_dfs):
    cross = cross_validation(runs, tt_folds, metrics, base_qry)
    stop_df.loc[ab] = cross[0]

In [14]:
stop_df.loc['$R$'] = base_df.round(4)

In [15]:
# stop_df = stop_df.rename(index={'nnphrase': 'phrase', 'nnphrase-exp': 'phrase-exp', 'phrase-exp-qry': 'phrase-qry', 'nnphrase-exp-qry': 'phrase-exp-qry'})
stop_df = stop_df.reindex(['$R$', 'phrase', 'phrase-exp', 'phrase-qry', 'phrase-exp-qry'])
stop_df

,recip_rank,err@20,recall_20,ndcg,rbp@0.80,unjudged@20
$R$,0.5686,0.3794,0.46,0.4613,0.3113,0
phrase,0.5636,0.3755,0.4620,0.4594,0.3100,0.6737$^{**}$
phrase-exp,0.5608,0.3794,0.4591,0.4601,0.3106,0.4632$^{**}$
phrase-qry,0.5605,0.3792,0.4584,0.4597,0.3097,0.4947$^{**}$
phrase-exp-qry,0.5609,0.3794,0.4579,0.4598,0.3095,0.4105$^{**}$


In [16]:
write_table('tables/ausnl-phrase', bold_max(stop_df.reindex(['$R$', 'phrase'])).rename(columns=metrics).drop('Unjudged@20',axis='columns').to_latex(escape=False))

In [17]:
write_table('tables/ausnl-phrase-exp', bold_max(stop_df).rename(columns=metrics).drop('Unjudged@20',axis='columns').to_latex(escape=False))